In [1]:
import random
class NTRU(object):
    def __init__(self, Para):
        self.p = 3
        if Para == 'ees1171':
            self.n = 1171
            self.q = 2048
            self.dg = 390
            self.df = 106
            self.seed = 400
        elif Para == 'ees1087':
            self.n = 1087
            self.q = 2048
            self.dg = 362
            self.df = 120
            self.seed = 400  
        elif Para == 'ees1499':
            self.n = 1499
            self.q = 2048
            self.dg = 499
            self.df = 79
            self.seed = 400     
        elif Para == 'demo':
            self.n = 13
            self.q = 2048
            self.dg = 3
            self.df = 3
            self.seed = 5
        elif Para == 'hps2048509' : 
            self.n = 509
            self.q = 2048
            self.dg = 127
            self.df = 127
            self.seed = 300
        elif Para == 'hps2048677' : 
            self.n = 677
            self.q = 2048
            self.dg = 127
            self.df = 127
            self.seed = 300
        elif Para == 'hps2048821' :
            self.n = 821
            self.q = 2048
            self.dg = 127
            self.df = 127
            self.seed = 300

In [7]:
ntru = NTRU('hps2048677')

dim = (ntru.n/2).floor() + 1
ll = 2

In [8]:
def modCoeffs(f, pp):
    clist = f.list()
    p2 = pp/2.floor()
    for i in range(len(clist)):
        clist[i] = Integer(clist[i]) % Integer(pp)
        if clist[i] > p2:
            clist[i] -= pp
    return R(clist)

def modCoeffs_list(clist, pp):
    p2 = pp/2.floor()
    for i in range(len(clist)):
        clist[i] = Integer(clist[i]) % Integer(pp)
        if clist[i] > p2:
            clist[i] -= pp
    return clist

def gen_f(df):
    L_0 = [0] * (ntru.n - 2 * df - 1)
    L_1 = (df + 1) * [1]
    L_2 = (df) * [-1]
    L = L_0 + L_1 + L_2
    for i in range(randint(5,10)):
        random.shuffle(L)
    poly = R(L) % (x^ntru.n - 1)
    return poly

def gen_g(dg):
    L_0 = [0] * (ntru.n - 2 * dg)
    L_1 = (dg) * [1]
    L_2 = (dg) * [-1]
    L = L_0 + L_1 + L_2
    for i in range(randint(5,10)):
        random.shuffle(L)
    poly = R(L) % (x^ntru.n - 1)
    return poly

def __inv_poly_mod2__(poly):
    k = 0; b = 1; c = 0 * x
    f = poly; g = x^ntru.n-1
    f = modCoeffs(f, 2)
    res = False
    while True:
        while f(0) == 0 and not f.is_zero():
            f = f.shift(-1)
            c = c.shift(1)
            c = modCoeffs(c, 2)
            k += 1
        if f.is_one():
            e = (-k) % ntru.n
            retval = x^e * b 
            res = True
            break
        elif f.degree() == -1 or f.is_zero():
            break
        if f.degree() < g.degree():
            f, g = g, f
            b, c = c, b
        f = f + g
        b = b + c
        f = modCoeffs(f, 2)
        c = modCoeffs(c, 2)
    if res:
        retval = retval % (x^ntru.n - 1)
        retval = modCoeffs(retval, 2)
        return True, retval
    else:
        return False,0

def __inv_poly_mod_prime_pow__(poly):
    res,b = __inv_poly_mod2__(poly)
    if res:
        qr = 2
        while qr < ntru.q:
            qr = qr^2
            b = b * (2 - poly * b)
            b = b % (x^ntru.n - 1)
            b = modCoeffs(b, ntru.q)
        return True,b
    else:
        return False,0

def build(h_vector):
    H = matrix.circulant(h_vector).T   
    H = matrix(Integers(ntru.q),H)
    HH_Gram = H.T * H  
    c_coe = HH_Gram[0].list()
    mat_coe = []
    for i in range(dim):
        if i == 0 :
            mat_coe.append(c_coe[i])
        else :
            mat_coe.append(2 * c_coe[i])
    return mat_coe

R.<x> = PolynomialRing(Integers(ntru.q))
Q = R.quotient(x^ntru.n - 1)   


In [9]:
g_weight_temp = gen_g(ntru.dg).list()
g_weight = g_weight_temp.count(1) + g_weight_temp.count(-1)
f_weight_temp = gen_f(ntru.df).list()
f_weight = f_weight_temp.count(1) + f_weight_temp.count(-1)

In [10]:
def run_once() :

    # f = Q(gen_f(ntru.df))
    # f_q = f^(-1)
    # generate 1 f
    f = R(gen_f(ntru.df)) % (x^ntru.n - 1)
    [_, f_q] = __inv_poly_mod_prime_pow__(f)
    assert(Q(f * f_q) == 1)
    f_mat = matrix.circulant(Q(f.list()).list())
    f_conj = Q(f_mat[:,0].list())
    f = Q(f.list())
    f_q = Q(f_q.list())
    ffbar = (f * f_conj).list()
    
    # genrate (dim + ll) sample h and g
    HH_Gram_Folden = matrix(Integers(ntru.q), dim + ll, dim)
    cnt = 0
    while cnt < (dim + ll):
        h_test = Q(gen_g(ntru.dg) * f_q).list()
        HH_Gram_Folden[cnt] = build(h_test)
        if HH_Gram_Folden[cnt][0] % 2 == 0 :
            cnt = cnt + 1

    HH_Gram_Folden_2 = [[int(element) for element in row] for row in HH_Gram_Folden.rows()]
    HH_Gram_Folden_2 = [[element // 2 for element in row] for row in HH_Gram_Folden_2]


    u_col = (dim + ll) * [g_weight]
    for i in range(dim + ll) :
        u_col[i] = u_col[i] / 2

    for i in range(dim + ll) : 
        u_col[i] = u_col[i] - HH_Gram_Folden_2[i][0] * f_weight

    HH_Gram_Folden_2 = [row[1:] for row in HH_Gram_Folden_2]

    HH_Gram_Folden_2 = matrix(Integers(ntru.q/2), HH_Gram_Folden_2)
    u_mat = matrix(Integers(ntru.q/2),u_col).T

    result = HH_Gram_Folden_2.solve_right(u_mat)
    result = [f_weight] +  result.list()
    
    result = modCoeffs_list(result, ntru.q/2)
    ffbar_test = modCoeffs_list(ffbar, ntru.q)
    return result[0 : (dim)] == ffbar_test[0 : (dim)], result, ffbar_test

In [11]:
import time

TEST_NUM = 100
cnt = 0

start = time.time()
for _ in range(TEST_NUM) :
    ok, result, ffbar_test = run_once()
    if (ok == False) :
        print("bad situations ")
        print(result)
        print(ffbar_test)
        cnt = cnt + 1
end = time.time()

print("run time ", (end - start), " seconds")
print("run time per test ", (end - start) / TEST_NUM, " seconds")
print("error count ", cnt)

bad situations 
[255, 264, 500, 493, -506, 244, -276, 5, 240, 2, -16, -235, 16, -506, -232, -11, 507, -272, 9, 6, -258, 262, -3, 510, -271, 271, 502, -5, 7, -246, -247, -509, 508, 247, -509, -506, 510, -7, -7, 508, 506, -7, -500, 260, 252, -3, -273, 263, -4, 15, 262, -259, 267, -502, 249, 494, 9, 498, -9, -2, -271, -243, 501, 17, 7, -2, -10, -507, 263, 238, 281, -256, 266, -255, 3, -502, -6, -501, 242, -510, 506, 277, 506, -261, 7, -252, 16, -2, -9, -241, 248, -271, 507, -255, 235, -250, 257, -274, -246, -249, -498, -256, -256, -261, 256, 501, -269, 506, 266, -11, 8, -12, -247, 248, 1, -260, 501, -500, 477, 509, 6, 272, -257, -258, 504, 231, 264, 251, -503, 8, 5, -505, 500, 8, -508, -507, -511, -266, -10, -265, -494, 235, 252, -8, -2, 7, 512, -508, 250, 254, -3, 268, -7, -254, -263, -2, -252, -6, -510, -260, -258, 5, 266, -2, 236, 1, -7, -273, 496, 6, 16, 512, 0, -253, -506, 258, -7, -9, -504, 246, 505, 261, -500, -505, 512, 1, 235, -247, 262, 501, 234, 508, -508, -503, 505, -8, 261, 2

run time  5218.780147075653  seconds
run time per test  52.18780147075653  seconds
error count  2
